In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import wooldridge as woo
import statsmodels.formula.api as smf
import scipy.stats as stats

# Exercise 12

We will now use the dataset 'state_aid'. It covers parts from the data used in the paper _The impact of state aid on the survival and financial viability of aided firms_ which you can find the dropbox folder 'papers'.


The data is in the Stata format '.dta'. The variables included are as follow. If you want to know more details, please look at the data description in the paper.

**Binary outcome variable**    
aid = dummy variable which is 1 if a firm has received state aid in a year and zero otherwise
	
**factors varying over firm and time**
ln_profit = logarithm of profit  
ln_fixed_liabilities = logarithm of fixed liability  
ln_current_liabilities = logarithm of current liabilities  
rev_emp = Revenue per employee  
altman_kat = cateogry of Altman z-score  
public = pulicly listed firm  
liquidity_ratio = liquidity ratio  
solvency_ratio = solvency ratio  
ln_employment = logarithm of number of employees  
size = firm size (small, medium, large, vlarge)  
altman_kat = categories of the Altman Z-score (1-3), see paper for details on categories.  
	
**Factors varying over country and time**  
unempl_3y = average unemployment rate last 3 years (per country)  
sub_gov_exp = ratio of subsidies to governmental expenditures (per country)  
real_gdp_per_cap = pct change of real gdp per capita (per country)    
nace_dig1 = first digit of main industry code of a firm according to NACE Rev.2 industry classification  

cid = country IDs  
year = years  

Firms that have received state aid are removed from the dataset in the years after they received the aid, so that the effect of state aid does not disturb the analysis.

We are interested in understanding the factors determining whether a firm receives state aid.

In [10]:
df = pd.read_stata('data/state_aid.dta')
df.head()

,year,liquidity_ratio,solvency_ratio,aid,sub_gov_exp,real_gdp_per_cap,unempl_3y,ln_profit,rev_emp,age,ln_age,ln_employment,ln_fixed_liabilities,ln_current_liabilities,public,altman_kat,nace_1dig,size,cid
0,2000,NaN,NaN,0,68.433328,3.1,4.0,NaN,NaN,21.0,3.044523,NaN,NaN,NaN,0.0,NaN,7.0,medium,AT
1,2001,NaN,NaN,0,68.169868,1.0,3.7,NaN,NaN,22.0,3.091043,NaN,NaN,NaN,0.0,NaN,7.0,medium,AT
2,2002,NaN,NaN,0,70.111311,1.2,3.8,NaN,NaN,23.0,3.135494,NaN,NaN,NaN,0.0,NaN,7.0,medium,AT
3,2003,NaN,NaN,0,70.671877,0.3,4.0,NaN,NaN,24.0,3.178054,NaN,NaN,NaN,0.0,NaN,7.0,medium,AT
4,2004,0.503,39.339,0,64.605007,2.1,4.5,7.315299,NaN,25.0,3.218876,NaN,8.198194,9.354279,0.0,2.0,7.0,medium,AT


In [11]:
df.describe()

,year,liquidity_ratio,solvency_ratio,aid,sub_gov_exp,real_gdp_per_cap,unempl_3y,ln_profit,rev_emp,age,ln_age,ln_employment,ln_fixed_liabilities,ln_current_liabilities,public,altman_kat,nace_1dig
count,2.388148e+06,1.361479e+06,1.365257e+06,2.388148e+06,2.223468e+06,2.228893e+06,2.228893e+06,1.366363e+06,1.178589e+06,2.379763e+06,2.295203e+06,1.186423e+06,1.371568e+06,1.369060e+06,2.384633e+06,1.349159e+06,2.388138e+06
mean,2.007001e+03,1.647078e+00,3.376706e+01,2.554281e-05,6.515370e+01,6.412648e-01,1.081189e+01,1.955888e+00,5.244034e+02,1.835753e+01,2.669579e+00,2.660417e+00,4.834971e+00,6.725017e+00,2.747895e-01,2.096575e+00,3.410451e+00
std,4.320252e+00,3.890890e+00,2.619113e+01,5.053926e-03,1.002479e+01,2.374210e+00,4.244861e+00,4.299615e+00,2.133581e+04,1.589013e+01,8.010735e-01,1.569365e+00,2.705094e+00,2.030857e+00,4.464082e-01,7.547300e-01,1.708083e+00
min,2.000000e+03,0.000000e+00,-1.000000e+02,0.000000e+00,1.455830e+01,-8.700000e+00,3.700000e+00,-1.570412e+01,-1.264303e+04,-1.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,-1.125526e+01,0.000000e+00,1.000000e+00,0.000000e+00
25%,2.003000e+03,6.360000e-01,1.538300e+01,0.000000e+00,5.315411e+01,-4.000000e-01,8.200000e+00,-1.007597e+00,9.635586e+01,9.000000e+00,2.302585e+00,1.609438e+00,3.253552e+00,5.382508e+00,0.000000e+00,2.000000e+00,2.000000e+00
50%,2.007000e+03,9.980000e-01,3.105600e+01,0.000000e+00,6.757085e+01,1.100000e+00,9.300000e+00,3.008452e+00,1.688554e+02,1.500000e+01,2.772589e+00,2.484907e+00,5.029384e+00,6.579833e+00,0.000000e+00,2.000000e+00,4.000000e+00
75%,2.011000e+03,1.540000e+00,5.097900e+01,0.000000e+00,7.409285e+01,1.800000e+00,1.130000e+01,4.910023e+00,3.323456e+02,2.400000e+01,3.178054e+00,3.583519e+00,6.556378e+00,7.982051e+00,1.000000e+00,3.000000e+00,4.000000e+00
max,2.014000e+03,9.995800e+01,1.000000e+02,1.000000e+00,8.131820e+01,1.060000e+01,2.410000e+01,1.714244e+01,1.371726e+07,2.009000e+03,7.605392e+00,1.325829e+01,1.921245e+01,2.038297e+01,1.000000e+00,3.000000e+00,9.000000e+00


**1.** What are the options to estimate such a model?

One can can use 3 models :
- linear probability model
- logit model
- probit model.

**2.** Estimate a linear probability model first. Include log_profit, ln_fixed_liabilities and ln_current_liabilities. Do the estimated parameters make sens? Are they significant? How do we interpret them? What are the drawbacks of an linear probability model? When do they matter?

In [12]:
reg_lin = smf.ols(formula='aid ~ ln_profit + ln_fixed_liabilities + ln_current_liabilities', data=df)
res_lin = reg_lin.fit()
b_lin = res_lin.params
res_lin.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    aid   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     112.8
Date:                Mon, 19 Sep 2022   Prob (F-statistic):           4.82e-73
Time:                        14:19:56   Log-Likelihood:             4.8828e+06
No. Observations:             1355480   AIC:                        -9.766e+06
Df Residuals:                 1355476   BIC:                        -9.766e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.0002   1.99e-05    -10.800      0.000      -0.000      -0.000
ln_profit              -1.819e-05   1.35e-06    -13.451      0.000   -2.08e-05   -1.55e-05
ln_fixed_liabilities    1.164e-06   2.71e-06      0.430      0.667   -4.14e-06    6.47e-06
ln_current_liabilities  4.279e-05   3.66e-06     11.684      0.000    3.56e-05       5e-05
==============================================================================
Omnibus:                  6107280.883   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   29772622228985.484
Skew:                         151.506   Prob(JB):                         0.00
Kurtosis:                   22960.774   Cond. No.                         32.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The parameter estimates from the linear probability model make intutively sense. A firm with higher liabilities and lower profits is more likely to receive state aide. However, the parameter estimates are very small. This also makes sens: most firms don'r receive state aid regadless of the movements of their prodits and liabilities.

The drawbacks are :
- we get results outside the interval $[0,1]$
- it creates heteroskodasticity

**3.** Add all other variables listed that vary over firm and time and re-estimate the model. Do the estimates make sens? Are they significant? Interpret them.

In [13]:
reg_lin = smf.ols(formula='aid ~ ln_profit + ln_fixed_liabilities + ln_current_liabilities + rev_emp + C(altman_kat) + C(public) + liquidity_ratio + solvency_ratio + ln_employment + size', data=df)
res_lin = reg_lin.fit()
b_lin = res_lin.params
res_lin.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    aid   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     45.48
Date:                Mon, 19 Sep 2022   Prob (F-statistic):          6.12e-118
Time:                        14:20:26   Log-Likelihood:             4.1118e+06
No. Observations:             1160823   AIC:                        -8.224e+06
Df Residuals:                 1160809   BIC:                        -8.223e+06
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               5.288e-05   4.28e-05      1.237      0.216   -3.09e-05       0.000
C(altman_kat)[T.2.0]   -6.231e-05   1.85e-05     -3.366      0.001   -9.86e-05    -2.6e-05
C(altman_kat)[T.3.0]   -6.915e-05   2.35e-05     -2.940      0.003      -0.000   -2.31e-05
C(public)[T.1.0]        3.673e-05   1.63e-05      2.258      0.024    4.85e-06    6.86e-05
size[T.medium]         -3.229e-05    1.9e-05     -1.703      0.089   -6.94e-05    4.87e-06
size[T.large]          -5.997e-05   3.16e-05     -1.897      0.058      -0.000    2.01e-06
size[T.vlarge]             0.0002   5.23e-05      3.226      0.001    6.62e-05       0.000
ln_profit              -1.763e-05   1.75e-06    -10.085      0.000   -2.11e-05   -1.42e-05
ln_fixed_liabilities   -1.597e-05   3.51e-06     -4.554      0.000   -2.28e-05    -9.1e-06
ln_current_liabilities -8.005e-06   7.22e-06     -1.109      0.267   -2.21e-05    6.14e-06
rev_emp                 4.902e-11   3.03e-10      0.162      0.872   -5.45e-10    6.44e-10
liquidity_ratio         3.287e-06   2.17e-06      1.515      0.130   -9.64e-07    7.54e-06
solvency_ratio         -1.535e-06   3.23e-07     -4.749      0.000   -2.17e-06   -9.02e-07
ln_employment              0.0001   7.36e-06     13.614      0.000    8.58e-05       0.000
==============================================================================
Omnibus:                  5135765.190   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   20013582350401.516
Skew:                         142.588   Prob(JB):                         0.00
Kurtosis:                   20342.597   Cond. No.                     2.19e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.19e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**4.** Add all factors varying over country and time now. Do the estimate make sense? Are they significant? Interpret them.

In [14]:
reg_lin = smf.ols(formula='aid ~ ln_profit + ln_fixed_liabilities + ln_current_liabilities + rev_emp + public + liquidity_ratio + solvency_ratio + ln_employment + size + altman_kat + unempl_3y + sub_gov_exp + real_gdp_per_cap + nace_1dig', data=df)
res_lin = reg_lin.fit()
b_lin = res_lin.params
res_lin.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    aid   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     38.94
Date:                Mon, 19 Sep 2022   Prob (F-statistic):          3.11e-122
Time:                        14:20:30   Log-Likelihood:             4.1051e+06
No. Observations:             1159164   AIC:                        -8.210e+06
Df Residuals:                 1159147   BIC:                        -8.210e+06
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept               4.729e-05   7.51e-05      0.630      0.529   -9.98e-05       0.000
size[T.medium]          -3.56e-05   1.91e-05     -1.867      0.062    -7.3e-05    1.77e-06
size[T.large]          -6.555e-05   3.17e-05     -2.066      0.039      -0.000   -3.36e-06
size[T.vlarge]             0.0002   5.26e-05      2.936      0.003    5.13e-05       0.000
ln_profit              -1.947e-05   1.77e-06    -11.012      0.000   -2.29e-05    -1.6e-05
ln_fixed_liabilities   -1.443e-05   3.55e-06     -4.060      0.000   -2.14e-05   -7.47e-06
ln_current_liabilities -3.383e-06   7.38e-06     -0.458      0.647   -1.79e-05    1.11e-05
rev_emp                 4.328e-11   3.04e-10      0.143      0.887   -5.52e-10    6.38e-10
public                  3.914e-05   1.63e-05      2.397      0.017    7.13e-06    7.12e-05
liquidity_ratio         3.712e-06   2.18e-06      1.705      0.088   -5.55e-07    7.98e-06
solvency_ratio         -1.284e-06   3.28e-07     -3.913      0.000   -1.93e-06   -6.41e-07
ln_employment           9.604e-05   7.54e-06     12.736      0.000    8.13e-05       0.000
altman_kat             -3.609e-05   1.19e-05     -3.043      0.002   -5.93e-05   -1.28e-05
unempl_3y               1.265e-06   1.66e-06      0.761      0.447   -1.99e-06    4.52e-06
sub_gov_exp            -7.285e-07   7.29e-07     -0.999      0.318   -2.16e-06    7.01e-07
real_gdp_per_cap         1.53e-05   2.81e-06      5.449      0.000    9.79e-06    2.08e-05
nace_1dig               8.191e-06   3.98e-06      2.058      0.040    3.89e-07     1.6e-05
==============================================================================
Omnibus:                  5127253.363   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   19925639808141.578
Skew:                         142.480   Prob(JB):                         0.00
Kurtosis:                   20312.376   Cond. No.                     2.67e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

It doesn't make sense...

**5.** Add firm size and re-estimate the model. Do the estimates make sense? Are they significant? Interpet them.

In [15]:
# ???

It still doesn't make sense.

**6.** Now control for the categorical variables determining the industry of a firm, the country in which its head office is located as well as the year. Why are these variables especially useful? What are they capturing? How do you have to interpret the coefficients?

In [16]:
reg_lin = smf.ols(formula='aid ~ ln_fixed_liabilities + ln_current_liabilities + ln_profit + liquidity_ratio + solvency_ratio + rev_emp + ln_age + sub_gov_exp + real_gdp_per_cap + unempl_3y + C(altman_kat) + C(public) + C(size) + C(nace_1dig)', data=df)
results_linear = reg_lin.fit()
results_linear.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    aid   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     20.41
Date:                Mon, 19 Sep 2022   Prob (F-statistic):           2.33e-88
Time:                        14:21:21   Log-Likelihood:             4.0740e+06
No. Observations:             1151472   AIC:                        -8.148e+06
Df Residuals:                 1151447   BIC:                        -8.148e+06
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.0003      0.000      1.771      0.077   -3.23e-05       0.001
C(altman_kat)[T.2.0]   -3.081e-05   1.87e-05     -1.646      0.100   -6.75e-05    5.87e-06
C(altman_kat)[T.3.0]   -1.448e-05   2.37e-05     -0.610      0.542    -6.1e-05    3.21e-05
C(public)[T.1.0]         3.85e-05   1.68e-05      2.295      0.022    5.63e-06    7.14e-05
C(size)[T.medium]      -1.519e-06   1.91e-05     -0.080      0.937   -3.89e-05    3.59e-05
C(size)[T.large]         7.16e-06   3.15e-05      0.227      0.820   -5.46e-05     6.9e-05
C(size)[T.vlarge]          0.0003   5.21e-05      5.541      0.000       0.000       0.000
C(nace_1dig)[T.1.0]       -0.0004      0.000     -2.815      0.005      -0.001      -0.000
C(nace_1dig)[T.2.0]       -0.0004      0.000     -2.704      0.007      -0.001      -0.000
C(nace_1dig)[T.3.0]       -0.0004      0.000     -2.499      0.012      -0.001   -8.49e-05
C(nace_1dig)[T.4.0]       -0.0004      0.000     -2.886      0.004      -0.001      -0.000
C(nace_1dig)[T.5.0]       -0.0004      0.000     -2.513      0.012      -0.001    -8.7e-05
C(nace_1dig)[T.6.0]       -0.0001      0.000     -0.550      0.582      -0.001       0.000
C(nace_1dig)[T.7.0]       -0.0004      0.000     -2.709      0.007      -0.001      -0.000
C(nace_1dig)[T.9.0]       -0.0003      0.000     -1.596      0.111      -0.001    6.46e-05
ln_fixed_liabilities   -4.643e-06   3.51e-06     -1.321      0.186   -1.15e-05    2.24e-06
ln_current_liabilities   3.43e-05   6.88e-06      4.985      0.000    2.08e-05    4.78e-05
ln_profit              -2.151e-05   1.78e-06    -12.076      0.000    -2.5e-05    -1.8e-05
liquidity_ratio          3.92e-06    2.2e-06      1.779      0.075      -4e-07    8.24e-06
solvency_ratio         -9.906e-07   3.38e-07     -2.930      0.003   -1.65e-06   -3.28e-07
rev_emp                 -1.81e-10   3.04e-10     -0.595      0.552   -7.77e-10    4.15e-10
ln_age                  1.572e-05   9.51e-06      1.654      0.098   -2.91e-06    3.44e-05
sub_gov_exp            -6.025e-07   7.37e-07     -0.818      0.413   -2.05e-06    8.41e-07
real_gdp_per_cap        2.176e-05   2.83e-06      7.680      0.000    1.62e-05    2.73e-05
unempl_3y               1.721e-06   1.67e-06      1.027      0.304   -1.56e-06       5e-06
==============================================================================
Omnibus:                  5088368.459   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):   19540324777001.414
Skew:                         142.030   Prob(JB):                         0.00
Kurtosis:                   20182.100   Cond. No.                     1.55e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix

The results are as expected. Large firms and public firms are more likely to receive state aid and firms with a lower Altman z-score value also have a higher probability to receive state aid.

**7.** Estimate the last (full) model in logit. How does the interpretation change?

In [17]:
reg_logit = smf.logit(formula='aid ~ ln_fixed_liabilities + ln_current_liabilities + ln_profit + liquidity_ratio + solvency_ratio + rev_emp + ln_age + sub_gov_exp + real_gdp_per_cap + unempl_3y + C(altman_kat) + C(public) + C(size) + C(nace_1dig)', data=df)
results_logit = reg_logit.fit()
results_logit.summary()

/opt/miniconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


Optimization terminated successfully.
         Current function value: 0.000408
         Iterations 18


/opt/miniconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    aid   No. Observations:              1151472
Model:                          Logit   Df Residuals:                  1151447
Method:                           MLE   Df Model:                           24
Date:                Mon, 19 Sep 2022   Pseudo R-squ.:                  0.2455
Time:                        14:22:21   Log-Likelihood:                -469.32
converged:                       True   LL-Null:                       -622.07
Covariance Type:            nonrobust   LLR p-value:                 1.310e-50
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                -11.2804      1.692     -6.667      0.000     -14.597      -7.964
C(altman_kat)[T.2.0]      -0.4112      0.360     -1.142      0.253      -1.117       0.294
C(altman_kat)[T.3.0]      -1.1595      0.661     -1.754      0.079      -2.455       0.136
C(public)[T.1.0]           0.7310      0.314      2.331      0.020       0.116       1.346
C(size)[T.medium]          0.8866      0.591      1.501      0.133      -0.271       2.045
C(size)[T.large]           1.0028      0.672      1.493      0.135      -0.314       2.319
C(size)[T.vlarge]          1.8344      0.839      2.187      0.029       0.191       3.478
C(nace_1dig)[T.1.0]       -2.3170      1.091     -2.124      0.034      -4.455      -0.179
C(nace_1dig)[T.2.0]       -2.1572      1.043     -2.069      0.039      -4.201      -0.113
C(nace_1dig)[T.3.0]       -1.7069      1.074     -1.589      0.112      -3.812       0.398
C(nace_1dig)[T.4.0]       -2.5718      1.064     -2.418      0.016      -4.657      -0.487
C(nace_1dig)[T.5.0]       -1.7426      1.081     -1.612      0.107      -3.862       0.377
C(nace_1dig)[T.6.0]       -1.2665      1.448     -0.874      0.382      -4.105       1.572
C(nace_1dig)[T.7.0]       -2.3686      1.132     -2.092      0.036      -4.588      -0.149
C(nace_1dig)[T.9.0]        0.2373      1.444      0.164      0.870      -2.593       3.068
ln_fixed_liabilities      -0.1144      0.056     -2.061      0.039      -0.223      -0.006
ln_current_liabilities     0.4420      0.117      3.783      0.000       0.213       0.671
ln_profit                 -0.1123      0.022     -5.018      0.000      -0.156      -0.068
liquidity_ratio           -0.4367      0.375     -1.164      0.244      -1.172       0.299
solvency_ratio            -0.0216      0.005     -4.431      0.000      -0.031      -0.012
rev_emp                   -0.0006      0.000     -3.922      0.000      -0.001      -0.000
ln_age                     0.3486      0.162      2.156      0.031       0.032       0.665
sub_gov_exp               -0.0114      0.013     -0.882      0.378      -0.037       0.014
real_gdp_per_cap           0.3972      0.057      7.023      0.000       0.286       0.508
unempl_3y                 -0.0036      0.043     -0.084      0.933      -0.088       0.081
==========================================================================================

Possibly complete quasi-separation: A fraction 0.93 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [18]:
reg_probit = smf.probit(formula='aid ~ ln_fixed_liabilities + ln_current_liabilities + ln_profit + liquidity_ratio + solvency_ratio + rev_emp + ln_age + sub_gov_exp + real_gdp_per_cap + unempl_3y + C(altman_kat) + C(public) + C(size) + C(nace_1dig)', data=df)
results_probit = reg_probit.fit()
results_probit.summary()

Optimization terminated successfully.
         Current function value: 0.000406
         Iterations 16


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Probit Regression Results                           
==============================================================================
Dep. Variable:                    aid   No. Observations:              1151472
Model:                         Probit   Df Residuals:                  1151447
Method:                           MLE   Df Model:                           24
Date:                Mon, 19 Sep 2022   Pseudo R-squ.:                  0.2491
Time:                        14:23:28   Log-Likelihood:                -467.14
converged:                       True   LL-Null:                       -622.07
Covariance Type:            nonrobust   LLR p-value:                 1.721e-51
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -4.3435      0.493     -8.812      0.000      -5.310      -3.377
C(altman_kat)[T.2.0]      -0.0889      0.099     -0.896      0.371      -0.284       0.106
C(altman_kat)[T.3.0]      -0.2182      0.166     -1.318      0.188      -0.543       0.106
C(public)[T.1.0]           0.2028      0.088      2.307      0.021       0.030       0.375
C(size)[T.medium]          0.1796      0.148      1.210      0.226      -0.111       0.470
C(size)[T.large]           0.1729      0.178      0.970      0.332      -0.176       0.522
C(size)[T.vlarge]          0.4044      0.233      1.737      0.082      -0.052       0.861
C(nace_1dig)[T.1.0]       -0.7719      0.326     -2.369      0.018      -1.411      -0.133
C(nace_1dig)[T.2.0]       -0.7064      0.312     -2.266      0.023      -1.317      -0.095
C(nace_1dig)[T.3.0]       -0.5495      0.321     -1.711      0.087      -1.179       0.080
C(nace_1dig)[T.4.0]       -0.7881      0.315     -2.501      0.012      -1.406      -0.170
C(nace_1dig)[T.5.0]       -0.5843      0.323     -1.809      0.070      -1.217       0.049
C(nace_1dig)[T.6.0]       -0.3194      0.434     -0.736      0.462      -1.170       0.531
C(nace_1dig)[T.7.0]       -0.7433      0.333     -2.234      0.025      -1.395      -0.091
C(nace_1dig)[T.9.0]       -0.0454      0.430     -0.105      0.916      -0.888       0.798
ln_fixed_liabilities      -0.0347      0.017     -2.079      0.038      -0.067      -0.002
ln_current_liabilities     0.1460      0.035      4.193      0.000       0.078       0.214
ln_profit                 -0.0347      0.006     -5.575      0.000      -0.047      -0.022
liquidity_ratio           -0.0837      0.092     -0.906      0.365      -0.265       0.097
solvency_ratio            -0.0061      0.002     -4.009      0.000      -0.009      -0.003
rev_emp                   -0.0003   7.57e-05     -4.453      0.000      -0.000      -0.000
ln_age                     0.0940      0.046      2.065      0.039       0.005       0.183
sub_gov_exp               -0.0016      0.004     -0.413      0.680      -0.009       0.006
real_gdp_per_cap           0.1154      0.017      6.719      0.000       0.082       0.149
unempl_3y                 -0.0066      0.013     -0.515      0.607      -0.032       0.018
==========================================================================================

Possibly complete quasi-separation: A fraction 0.93 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

**8.** What is the difference between a logit and probit model?

Both models are very similar but the function is not the same. The logit model uses the logistic function whereas the probit model uses the cumulative standard normal distribution function.

**9.** Compute the APE for the most refined model and compare it with the linear probability coefficients. Is the linear probability model a good approximation in this case?

- APE: the effect of x on y averaged across all cases in the sampe. This is obtained by caclulating the partial effect for all cases, and taking the average.- PEA (partial effect at the average): The effect of some x on y for a hypothetical case with sample averages for all x's. This is obtained by setting all x's at their sample mean and obtaining the solpe of y with respect to one of the x's.
- In OLS where the independent wariable is entered in a linear fashion (no squared or interaction terms), these are equivalent. In fact, it is an assumption of OLS that the partial effect of X does not vary accors x's.
- PEA and APE differ when we have squared or interaction terms in OLS, or when we use logistic, probit.

In [19]:
xb_linear = results_linear.predict()
xb_logit = results_logit.predict()
xb_probit = results_probit.predict()

predictions = pd.DataFrame({'linear_pred': xb_linear,
'logit_pred': xb_logit,
'probit_pred': xb_probit})
predictions.describe()

/opt/miniconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


,linear_pred,logit_pred,probit_pred
count,1.151472e+06,1.151472e+06,1.151472e+06
mean,4.950186e-05,4.950186e-05,4.916672e-05
std,1.450707e-04,8.896870e-04,9.200915e-04
min,-1.849135e-03,0.000000e+00,0.000000e+00
25%,-4.195560e-05,1.239785e-06,4.564047e-07
50%,2.096984e-05,4.995123e-06,2.908795e-06
75%,1.063841e-04,1.875073e-05,1.486239e-05
max,1.292647e-03,6.691788e-01,8.869479e-01


The logit and the probit model predict indidividual outcomes much better thant the linear probability model.

While the highest predicted probability for a firm to receive state is only 0.001 with OLS, it is 0.89 and 0.67 in the probit and logit models.

**10.** Estimate a model with ln_profit, ln_fixes_liabilities, ln_current_liabilities, altman_kat and firm size. Think about values of these variables that would make it very likely that a firm receives state aid and predict the outcome. Do the same for values which make it likely that the firm doesn't receive state aid. Predict the outcome again. Compare the outcome against the outcomes of a linear probability model.